In [ ]:
import numpy as np
import warpfield
import warpfield.utils

In [ ]:
# download some example data (see https://github.com/andreasmang/nirep)
!wget https://github.com/andreasmang/nirep/raw/refs/heads/master/nifti/na01.nii.gz
!wget https://github.com/andreasmang/nirep/raw/refs/heads/master/nifti/na15.nii.gz

In [ ]:
# load data
fixed, _ = warpfield.load_data("na01.nii.gz")
moving, _ = warpfield.load_data("na15.nii.gz")

In [ ]:
# create a basic recipe with one affine registration level and a default pre-filter:
recipe = warpfield.Recipe()
recipe.pre_filter.clip_thresh = 0  # clip DC background, if present

# affine level properties
recipe.levels[0].smooth.sigmas = [2.0, 2.0, 2.0] # raised sigma to slow convergence & dampen oscillations
recipe.levels[0].repeat = 10

# add non-rigid registration levels:
recipe.add_level(block_size=[128, 128, 128])
recipe.levels[-1].smooth.sigmas = [1.0, 1.0, 1.0]
recipe.levels[-1].repeat = 5

recipe.add_level(block_size=[64, 64, 64])
recipe.levels[-1].block_stride = 0.5
recipe.levels[-1].smooth.sigmas = [1.0, 1.0, 1.0]
recipe.levels[-1].repeat = 5

recipe.add_level(block_size=[32, 32, 32])
recipe.levels[-1].block_stride = 0.5
recipe.levels[-1].smooth.sigmas = [2.0, 2.0, 2.0]
recipe.levels[-1].repeat = 10

In [ ]:
# register (without video)
moving_reg, warpmap, _ = warpfield.register_volumes_with_video(fixed, moving, recipe)

In [ ]:
# register (with MIP video)
video_path = "output.mp4"
units_per_voxel = [1,1,1]
moving_reg, warpmap, _ = warpfield.register_volumes_with_video(fixed, moving, recipe, video_fn=video_path, units_per_voxel=units_per_voxel)
warpfield.utils.showvid(video_path, embed=True)

In [ ]:
# register (with slice video)
video_path = "output.mp4"
vmax = np.percentile(recipe.pre_filter(fixed), 99)
callback = warpfield.utils.mosaic_callback(vmax=vmax, axis=1, num_slices=9)
moving_reg, warpmap, _ = warpfield.register_volumes_with_video(
    fixed, moving, recipe, video_fn=video_path, callback=callback, units_per_voxel=units_per_voxel
)
warpfield.utils.showvid(video_path, embed=True)